In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
import json
import time
import torch
import gc
from ultralytics import YOLO
from pathlib import Path

# Tyhjennä GPU-muisti alkuun
torch.cuda.empty_cache()

# Dataset-konfiguraatio
data_config = 'config/materials.yaml'

# Mallit ja niiden asetukset (YOLO11x pienemmällä batchilla)
models = {
    'YOLO11x': {'weights': 'yolo11x.pt', 'batch': 8},
    'YOLO11s': {'weights': 'yolo11s.pt', 'batch': 16},
    'YOLO11n': {'weights': 'yolo11n.pt', 'batch': 16}
}

# Tallennetaan tulokset
results_summary = {}

# Koulutus- ja validointisilmukka
for name, model_info in models.items():
    print(f"\n🚀 Koulutetaan malli: {name}")
    weights = model_info['weights']
    batch_size = model_info['batch']
    
    # Luo uusi YOLO-malli
    model = YOLO(weights)

    # Aloita ajanotto
    start_time = time.time()

    # Koulutus
    model.train(data=data_config, epochs=50, batch=batch_size)

    # Ajanotto loppuu
    train_duration = time.time() - start_time

    # Validointi
    results = model.val(data=data_config)
    metrics = results.box

    # Tallennetaan tulokset
    results_summary[name] = {
        'mAP50': round(metrics.map50, 4),
        'mAP50-95': round(metrics.map, 4),
        'precision': round(metrics.p.mean(), 4),
        'recall': round(metrics.r.mean(), 4),
        'train_time_sec': round(train_duration, 1)
    }

    # 🧹 Tyhjennä muisti ennen seuraavaa mallia
    del model
    gc.collect()
    torch.cuda.empty_cache()

# Tallenna tulokset tiedostoon
output_file = Path("mallivertailu_tulokset_11x_11s_11n.json")
with open(output_file, "w") as f:
    json.dump(results_summary, f, indent=4)

# Tulosta vertailu
print("\n📊 Vertailun tulokset (YOLO11x, 11s, 11n):")
print(f"{'Malli':<10} {'mAP50':>8} {'mAP50-95':>10} {'Prec.':>8} {'Recall':>8} {'Time (s)':>10}")
print("-" * 60)
for model_name, metrics in results_summary.items():
    print(f"{model_name:<10} {metrics['mAP50']:>8} {metrics['mAP50-95']:>10} {metrics['precision']:>8} {metrics['recall']:>8} {metrics['train_time_sec']:>10}")


In [ ]:
%matplotlib inline
import json
import pandas as pd
import matplotlib.pyplot as plt

# 🟡 1. Lue tallennetut tulokset tiedostosta
with open("mallivertailu_tulokset_11x_11s_11n.json", "r") as f:
    results = json.load(f)

# 🟡 2. Muodosta DataFrame
df = pd.DataFrame(results).T  # Transponoidaan niin että mallit ovat riveillä

# 🟡 3. Piirrä pylväsdiagrammit
for metric in df.columns:
    plt.figure(figsize=(6, 4))
    df[metric].plot(kind='bar', title=f'Malli vs {metric}', ylabel=metric, xlabel='Malli', legend=False)
    plt.xticks(rotation=0)
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()
